# CAL01原始数据特征提取

## 1、原始数据清理

#### 时间清理

## 2、特征提取

#### 原始数据时间筛选

#### 特征提取统计计算 - 改名

In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import scipy.stats as st
from datetime import datetime,date,timedelta
import time
import collections

In [2]:
### 定义的是核心按钮

jk_appid=['jkwx', 'jkand', 'jkand2', 'jkandroid', 'jkios', 'yzand1', 'jkiosspeed', 'jkiossolar','jkioslunar',
         'jkmjand_1','jkmjios_1','jkiosxjjsd','jkiosjqb','jkiosxysjd','jkiosjqd','jkiosxejsd','jkiossjdjk',
         'jkiosxjjk','jkiospphd','jkiosjyq','jkiosxedk','jkandjqb','jkandxedk','jkandpphd','jkori','jkoriand2','jkoriios']
jk_appid_and = [ 'jkand', 'jkand2', 'jkandroid','yzand1','jkmjand_1','jkandjqb','jkandxedk','jkandpphd','jkoriand2']
jk_appid_ios = [ 'jkios','jkiosspeed', 'jkiossolar','jkioslunar','jkmjios_1','jkiosxjjsd','jkiosjqb','jkiosxysjd','jkiosjqd','jkiosxejsd'
                ,'jkiossjdjk','jkiosxjjk','jkiospphd','jkiosjyq','jkiosxedk','jkoriios']

In [3]:
cal01 = pd.read_csv('./data_mid/20171122_data_cal01.201706_201708.csv',encoding="utf-8")

In [4]:
print(len(cal01.ugid.unique()))

338


In [5]:
cal01.columns

Index(['ugid', 'zuid', 'appid', 'mid', 'utime', 'ctime', 'count', 'date',
       'types', 'duration', 'name', 'number', 'remote_addr'],
      dtype='object')

In [6]:
cal01 = cal01[['ugid', 'zuid', 'appid', 'mid', 'utime', 'ctime', 'count', 'date',
       'types', 'duration', 'name', 'number' ,'remote_addr']]

In [7]:
# 提取申请时间
label = pd.read_csv('./data_raw/APP_5K_20171120_ADD_UGID.csv')
# label = label.drop(['month','user_gid','last_decision_dt','SamplingWeight'],axis=1).rename(columns={'create_time':'last_decision_tm'})
label = label[['ugid','create_time']].rename(columns={'create_time':'last_decision_tm'})

In [8]:
label['last_decision_tm'] = label['last_decision_tm'].apply(lambda x :str(x).split('.')[0])

In [9]:
feature01 = label[['ugid']].drop_duplicates()

In [10]:
%%time

# 转换申请时间格式

label['last_decision_tm'] = label['last_decision_tm'].replace('DEC', '-12-',regex=True).\
replace('NOV', '-11-',regex=True).\
replace('OCT', '-10-',regex=True).\
replace('SEP', '-09-',regex=True).\
replace('AUG', '-08-',regex=True).\
replace('JAN', '-01-',regex=True).\
replace('MAY', '-05-',regex=True).\
replace('FEB', '-02-',regex=True).\
replace('MAR', '-03-',regex=True).\
replace('APR', '-04-',regex=True).\
replace('JUN', '-06-',regex=True)

label['last_decision_tm'] = label['last_decision_tm'].apply(lambda x :datetime.strptime(x, '%d-%m-%Y:%H:%M:%S'))
label['last_decision_tm'] = pd.to_datetime(label['last_decision_tm'])

CPU times: user 164 ms, sys: 4 ms, total: 168 ms
Wall time: 167 ms


In [11]:
cal01=pd.merge(label,cal01,on='ugid',how='left')

In [12]:
#%%time

# 异常值处理
cal01['ctime'] = cal01['ctime'].replace('null','-1').astype('float')
cal01['ctime'] = cal01['ctime'].replace(np.nan,'-1').astype('float')
cal01['ctime'] = cal01['ctime'].replace(np.inf,'-1').astype('float')
cal01['date'] = cal01['date'].replace('null','-1').astype('float')
cal01['date'] = cal01['date'].replace(np.nan,'-1').astype('float')
cal01['date'] = cal01['date'].replace(np.inf,'-1').astype('float')

In [13]:
# 清理时间 & 时间筛选 & appid筛选
cal01=cal01[(cal01.date==cal01.date) & (cal01['date']!=-1)]
cal01=cal01[((cal01.ctime==cal01.ctime) & (cal01['date']!=-1))]
cal01=cal01[((cal01.ugid==cal01.ugid) & (cal01['ugid']!=-1))]
cal01=cal01[cal01.appid.isin(jk_appid)]
cal01.shape

(321107, 14)

In [14]:
cal01['ctime'] = cal01.ctime.apply(lambda x: x*1000 if x<1000000000 else x)

In [15]:
#转换时间格式
#%%time

converDate=lambda x:datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')
cal01['active_day']=cal01.date.apply(lambda x:datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d'))
cal01['date']=cal01.date.apply(converDate)
cal01['ctime']=cal01.ctime.apply(converDate)

In [16]:
cal01['last_decision_tm'] = cal01.last_decision_tm.apply(lambda x:str(x).split('.')[0])

In [17]:
total_num_cal = cal01.groupby('ugid').duration.count().reset_index()

In [18]:
cal01['date'] = cal01.date.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [19]:
cal01['ctime'] = cal01.ctime.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [20]:
cal01['active_day'] = cal01.active_day.apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))

In [21]:
cal01['last_decision_tm'] = cal01.last_decision_tm.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [22]:

feature01 = cal01[(cal01.date + timedelta(days=60) > cal01.last_decision_tm) &  (cal01.date<cal01.last_decision_tm) ].reset_index(drop=True)
feature01 = feature01[['ugid']].drop_duplicates().reset_index()
feature01.shape

(245, 2)

In [23]:
def total_num_calls(feature01,df,days):
    '''
    输入一个dataframe，计算每个用户在统计时间内的通话总次数，包括通话总次数，通话总次数-主叫，通话总次数-被叫,通话总次数-主叫占比，通话总次数-被叫占比
    '''
    #df = df[(df.date+ timedelta(period) > cal01.last_decision_tm) &  (cal01.date<cal01.last_decision_tm) ].reset_index(drop=True)
    
    callcnt_agg = df.groupby('ugid').duration.count().reset_index()
    callcnt_agg_out = df[df.types == 2].groupby('ugid').duration.count().reset_index()
    callcnt_agg_in = df[df.types == 1].groupby('ugid').duration.count().reset_index()
    callcnt_agg_other = df[df.types == 3].groupby('ugid').duration.count().reset_index()
    if days == 30:
        callcnt_agg.columns = ['ugid', 'last1m_callcnt_agg']
        callcnt_agg_out.columns = ['ugid', 'last1m_callcnt_agg_out']
        callcnt_agg_in.columns = ['ugid', 'last1m_callcnt_agg_in'] 
        callcnt_agg_other.columns = ['ugid', 'last1m_callcnt_agg_other']
    elif days == 60:
        callcnt_agg.columns = ['ugid', 'last2m_callcnt_agg']
        callcnt_agg_out.columns = ['ugid', 'last2m_callcnt_agg_out']
        callcnt_agg_in.columns = ['ugid', 'last2m_callcnt_agg_in']
        callcnt_agg_other.columns = ['ugid', 'last2m_callcnt_agg_other']
    
    feature01 = pd.merge(feature01, callcnt_agg, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, callcnt_agg_out, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, callcnt_agg_in, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, callcnt_agg_other, how = 'left', on = 'ugid')
    return(feature01)

In [24]:
def uniq_phone_number(feature01,df,days):
    '''
    输入一个dataframe，计算每个用户在统计时间内的去重号码数，包括主叫，被叫
    '''
    tmp = df[['ugid', 'number','types']].drop_duplicates().reset_index()
    clnnum_cnt = tmp.groupby('ugid').number.count().reset_index()
    clnnum_cnt_out = tmp[tmp.types == 2].groupby('ugid').number.count().reset_index()
    clnnum_cnt_in = tmp[tmp.types == 1].groupby('ugid').number.count().reset_index()
    clnnum_cnt_other = tmp[tmp.types == 3].groupby('ugid').number.count().reset_index()
    if days == 30:
        clnnum_cnt.columns = ['ugid', 'last1m_clnnum_cnt']
        clnnum_cnt_out.columns = ['ugid', 'last1m_clnnum_cnt_out']
        clnnum_cnt_in.columns = ['ugid', 'last1m_clnnum_cnt_in']
        clnnum_cnt_other.columns = ['ugid', 'last1m_clnnum_cnt_other']
    elif days == 60:
        clnnum_cnt.columns = ['ugid', 'last2m_clnnum_cnt']
        clnnum_cnt_out.columns = ['ugid', 'last2m_clnnum_cnt_out']
        clnnum_cnt_in.columns = ['ugid', 'last2m_clnnum_cnt_in']
        clnnum_cnt_other.columns = ['ugid', 'last2m_clnnum_cnt_other']
    
    feature01 = pd.merge(feature01, clnnum_cnt, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, clnnum_cnt_out, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, clnnum_cnt_in, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, clnnum_cnt_other, how = 'left', on = 'ugid')
    return(feature01)

In [25]:
def total_time_call(feature01,df,days):
    '''
    输入一个dataframe，计算每个用户在统计时间内的通话总时长，包括通话总时长-主叫，通话总时长-被叫
    '''
    calltime_agg = df.groupby('ugid').duration.sum().reset_index()
    calltime_agg_out = df[df.types == 2].groupby('ugid').duration.sum().reset_index()
    calltime_agg_in = df[df.types == 1].groupby('ugid').duration.sum().reset_index()
    calltime_agg_other = df[df.types == 3].groupby('ugid').duration.sum().reset_index()
    if days == 30:
        calltime_agg.columns = ['ugid', 'last1m_calltime_agg']
        calltime_agg_out.columns = ['ugid', 'last1m_calltime_agg_out']
        calltime_agg_in.columns = ['ugid', 'last1m_calltime_agg_in']
        calltime_agg_other.columns = ['ugid', 'last1m_calltime_agg_other']
    elif days == 60:
        calltime_agg.columns = ['ugid', 'last2m_calltime_agg']
        calltime_agg_out.columns = ['ugid', 'last2m_calltime_agg_out']
        calltime_agg_in.columns = ['ugid', 'last2m_calltime_agg_in']
        calltime_agg_other.columns = ['ugid', 'last2m_calltime_agg_other']

    feature01 = pd.merge(feature01, calltime_agg, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, calltime_agg_out, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, calltime_agg_in, how = 'left', on = 'ugid')
    feature01 = pd.merge(feature01, calltime_agg_other, how = 'left', on = 'ugid')
    return(feature01)

In [26]:
def sort_data(df):
    '''
    对输入的数据的每一列都按升序排列
    '''
    data = df.reset_index()
    data = data[['ugid','date_first','date_last']]
    data_1 = data[['ugid','date_first']]
    data_2 = data[['ugid','date_last']]
    data_1 = data_1.sort_values(['date_first'], ascending=True)
    data_1 = data_1.reset_index()
    data_1 = data_1[['ugid','date_first']]
    data_2 = data_2.sort_values(['date_last'], ascending=True)
    data_2 = data_2.reset_index()
    data_2 = data_2[['ugid','date_last']]
    data_3 = pd.concat([data_1,data_2],axis = 1)
    data_0 = data_3[data_3['date_first'] < data_3['date_last']]
    return(data_0.shape[0])

In [27]:
def four_consecutive_days_no_call_pre(df,days):
    '''
    输入一个某个统计时间段（30天或60天）包含ugid和date的数据集，在date列上分别增加统计时间的开始和截至时间，
    生成新的两列数据（这两列数据横向对比即可标记是否连续四天未通话）。
    其中date_first列加上了4天，与date_last列比较。date_first小于date_last即为连续四天无通话记录。
    '''
    data = df[['ugid','last_decision_tm']].drop_duplicates().reset_index()
    data['date_last'] = data.last_decision_tm.apply(lambda x:str(x).split('.')[0])
    data = data[['ugid','date_last']]
    data['date_last'] = data.date_last.apply(lambda x:datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S'))
    data['date_first'] = data.date_last.apply(lambda x:x - timedelta(days))
    data = data[['ugid','date_first','date_last']]
    
    
    df_ = df[['ugid','date']].reset_index()
    df_ = df_[['ugid','date']]
    df_.columns = ['ugid','date_first']
    df_['date_last'] = df_['date_first']
    
    df_ = df_.append(data)
    df_['date_first'] = df_.date_first.apply(lambda x:x+timedelta(days=4))
    df_ = df_.reset_index()
    df_ = df_[['ugid','date_first','date_last']]
    
    df_4 = df_.groupby('ugid').apply(lambda row:sort_data(row))
    #df_4 = df_4[df_4['date_first'] + timedelta(days=4) < df_4['date_last']]
    
    return(df_4)

In [28]:
def four_consecutive_days_no_call(feature01,df,days):
    if days == 30:
        data = four_consecutive_days_no_call_pre(df,days=30)
        data_pd = pd.DataFrame(data)
        data_pd = data_pd.reset_index()
        data_pd.columns = ['ugid','last1m_callcnt_void4cnt']
        feature01 = pd.merge(feature01,data_pd,how = 'left',on = 'ugid')
    elif days == 60:
        data = four_consecutive_days_no_call_pre(df,days=60)
        data_pd = pd.DataFrame(data)
        data_pd = data_pd.reset_index()
        data_pd.columns = ['ugid','last2m_callcnt_void4cnt']
        feature01 = pd.merge(feature01,data_pd,how = 'left',on = 'ugid')
    return(feature01)

In [31]:
def dayBaseProportion(feature01,df,days):
    '''
    输入一个dataframe,生成所有daybase占比/平均值统计特征
    '''
    if days == 30:
        feature01['last1m_callcnt_rate_out'] = feature01['last1m_callcnt_agg_out'] / feature01['last1m_callcnt_agg']  #近1个月的主叫次数占比
        feature01['last1m_callcnt_rate_out'] = feature01['last1m_callcnt_rate_out'].astype('float')
        feature01['last1m_callcnt_rate_in'] = feature01['last1m_callcnt_agg_in'] / feature01['last1m_callcnt_agg']
        feature01['last1m_callcnt_rate_in'] = feature01['last1m_callcnt_rate_in'].astype('float')
        feature01['last1m_callcnt_rate_other'] = feature01['last1m_callcnt_agg_other'] / feature01['last1m_callcnt_agg']
        feature01['last1m_callcnt_rate_other'] = feature01['last1m_callcnt_rate_other'].astype('float')
    
        feature01['last1m_calltime_rate_out'] = feature01['last1m_calltime_agg_out'] / feature01['last1m_calltime_agg']  #近1个月的主叫时长占比
        feature01['last1m_calltime_rate_out'] = feature01['last1m_calltime_rate_out'].astype('float')
        feature01['last1m_calltime_rate_in'] = feature01['last1m_calltime_agg_in'] / feature01['last1m_calltime_agg']
        feature01['last1m_calltime_rate_in'] = feature01['last1m_calltime_rate_in'].astype('float')
        feature01['last1m_calltime_rate_other'] = feature01['last1m_calltime_agg_other'] / feature01['last1m_calltime_agg']
        feature01['last1m_calltime_rate_other'] = feature01['last1m_calltime_rate_other'].astype('float')
    
        feature01['last1m_calltime_avg'] = feature01['last1m_calltime_agg']/feature01['last1m_callcnt_agg']  #近1个月的平均通话时长
        feature01['last1m_calltime_avg'] = feature01['last1m_calltime_avg'].astype('float')
        feature01['last1m_calltime_avg_out'] = feature01['last1m_calltime_agg_out'] / feature01['last1m_callcnt_agg_out']
        feature01['last1m_calltime_avg_out'] = feature01['last1m_calltime_avg_out'].astype('float')
        feature01['last1m_calltime_avg_in'] = feature01['last1m_calltime_agg_in'] / feature01['last1m_callcnt_agg_in']
        feature01['last1m_calltime_avg_in'] = feature01['last1m_calltime_avg_in'].astype('float')
        feature01['last1m_calltime_avg_other'] = feature01['last1m_calltime_agg_other'] / feature01['last1m_callcnt_agg_other']
        feature01['last1m_calltime_avg_other'] = feature01['last1m_calltime_avg_other'].astype('float')
    elif days == 60:
        feature01['last2m_callcnt_rate_out'] = feature01['last2m_callcnt_agg_out'] / feature01['last2m_callcnt_agg']  
        feature01['last2m_callcnt_rate_out'] = feature01['last2m_callcnt_rate_out'].astype('float')
        feature01['last2m_callcnt_rate_in'] = feature01['last2m_callcnt_agg_in'] / feature01['last2m_callcnt_agg']
        feature01['last2m_callcnt_rate_in'] = feature01['last2m_callcnt_rate_in'].astype('float')
        feature01['last2m_callcnt_rate_other'] = feature01['last2m_callcnt_agg_other'] / feature01['last2m_callcnt_agg']
        feature01['last2m_callcnt_rate_other'] = feature01['last2m_callcnt_rate_other'].astype('float')
    
        feature01['last2m_calltime_rate_out'] = feature01['last2m_calltime_agg_out'] / feature01['last2m_calltime_agg']  
        feature01['last2m_calltime_rate_out'] = feature01['last2m_calltime_rate_out'].astype('float')
        feature01['last2m_calltime_rate_in'] = feature01['last2m_calltime_agg_in'] / feature01['last2m_calltime_agg']
        feature01['last2m_calltime_rate_in'] = feature01['last2m_calltime_rate_in'].astype('float')
        feature01['last2m_calltime_rate_other'] = feature01['last2m_calltime_agg_other'] / feature01['last2m_calltime_agg']
        feature01['last2m_calltime_rate_other'] = feature01['last2m_calltime_rate_other'].astype('float')
    
        feature01['last2m_calltime_avg'] = feature01['last2m_calltime_agg'] / feature01['last2m_callcnt_agg']  
        feature01['last2m_calltime_avg'] = feature01['last2m_calltime_avg'].astype('float')
        feature01['last2m_calltime_avg_out'] = feature01['last2m_calltime_agg_out'] / feature01['last2m_callcnt_agg_out']
        feature01['last2m_calltime_avg_out'] = feature01['last2m_calltime_avg_out'].astype('float')
        feature01['last2m_calltime_avg_in'] = feature01['last2m_calltime_agg_in'] / feature01['last2m_callcnt_agg_in']
        feature01['last2m_calltime_avg_in'] = feature01['last2m_calltime_avg_in'].astype('float')
        feature01['last2m_calltime_avg_other'] = feature01['last2m_calltime_agg_other'] / feature01['last2m_callcnt_agg_other']
        feature01['last2m_calltime_avg_other'] = feature01['last2m_calltime_avg_other'].astype('float')
    return(feature01)
        

In [32]:
def getDayBasefeature(feature01,df,days):
    feature01 = total_num_calls(feature01,df,days)    #通话总次数类特征 3*2
    feature01 = uniq_phone_number(feature01,df,days)  #去重号码类特征3*2
    feature01 = total_time_call(feature01,df,days)    #通话总时长类特征3*2
    feature01 = dayBaseProportion(feature01,df,days)  #平均值，占比类特征7*2
    feature01 = four_consecutive_days_no_call(feature01,df,days)   #连续四天无通话记录1*2
    return(feature01)

In [33]:
def weekday_active_days(feature01,df,days):
    '''
    输入一个dataframe，计算每个用户再统计时间内的活跃天数
    '''
    tmp = df[['ugid', 'active_day']].drop_duplicates()
    active_dtotal_wrk = tmp.groupby('ugid').active_day.count().reset_index()
    if days == 30:
        active_dtotal_wrk.columns = ['ugid', 'last1m_active_dtotal_wrk']      #近一个月活跃的工作日总数
    elif days == 60:
        active_dtotal_wrk.columns = ['ugid', 'last2m_active_dtotal_wrk']
        
    feature01 = pd.merge(feature01, active_dtotal_wrk, how = 'left', on = 'ugid')
    return(feature01)

In [34]:
def weekend_active_days(feature01,df,days):
    '''
    输入一个dataframe，计算每个用户在统计时间内的活跃天数
    '''
    df = df[['ugid', 'active_day']].drop_duplicates()
    active_dtotal_wkd = df.groupby('ugid').active_day.count().reset_index()
    if days == 30:
        active_dtotal_wkd.columns = ['ugid', 'last1m_active_dtotal_wkd']
    elif days == 60:
        active_dtotal_wkd.columns = ['ugid', 'last2m_active_dtotal_wkd']
        
    feature01 = pd.merge(feature01, active_dtotal_wkd, how = 'left', on = 'ugid')
    return(feature01)

In [35]:
def weekdayBaseSDK(feature01,df,days):
    '''
    输入一个dataframe，生成所有weekBase统计特征，包括通话总时长，通话总次数，去重号码数，通话平均时长
    '''
    callcnt_agg_wrk = df.groupby('ugid').duration.count().reset_index()     #工作日通话总次数
    calltime_agg_wrk = df.groupby('ugid').duration.sum().reset_index()    #工作日通话总时长
    tmp = df[['ugid', 'number']].drop_duplicates().reset_index() 
    clncnt_dtotal_wrk = tmp.groupby('ugid').number.count().reset_index()   #工作日通话去重号码数
    if days == 30:
        callcnt_agg_wrk.columns = ['ugid', 'last1m_callcnt_agg_wrk']
        feature01 = pd.merge(feature01, callcnt_agg_wrk, how = 'left', on = 'ugid')
        calltime_agg_wrk.columns = ['ugid', 'last1m_calltime_agg_wrk']
        feature01 = pd.merge(feature01, calltime_agg_wrk, how = 'left', on = 'ugid')
        clncnt_dtotal_wrk.columns = ['ugid', 'last1m_clncnt_dtotal_wrk']
        feature01 = pd.merge(feature01, clncnt_dtotal_wrk, how = 'left', on = 'ugid')
        feature01['last1m_calltime_avg_wrk'] = feature01['last1m_calltime_agg_wrk']/feature01['last1m_callcnt_agg_wrk']   #平均通话时长
    elif days == 60:
        callcnt_agg_wrk.columns = ['ugid', 'last2m_callcnt_agg_wrk']
        feature01 = pd.merge(feature01, callcnt_agg_wrk, how = 'left', on = 'ugid')
        calltime_agg_wrk.columns = ['ugid', 'last2m_calltime_agg_wrk']
        feature01 = pd.merge(feature01, calltime_agg_wrk, how = 'left', on = 'ugid')
        clncnt_dtotal_wrk.columns = ['ugid', 'last2m_clncnt_dtotal_wrk']
        feature01 = pd.merge(feature01, clncnt_dtotal_wrk, how = 'left', on = 'ugid')
        feature01['last2m_calltime_avg_wrk'] = feature01['last2m_calltime_agg_wrk']/feature01['last2m_callcnt_agg_wrk']
    return(feature01)
        

In [36]:
def weekendBaseSDK(feature01,df,days):
    '''
    输入一个dataframe，生成所有weekBase统计特征，包括通话总时长，通话总次数，去重号码数
    '''
    callcnt_agg_wkd = df.groupby('ugid').duration.count().reset_index()                         #休息日通话总次数
    calltime_agg_wkd = df.groupby('ugid').duration.sum().reset_index()                          #休息日通话总时长
    tmp = df[['ugid', 'number']].drop_duplicates().reset_index()                         #休息日通话去重号码数
    clncnt_dtotal_wkd = tmp.groupby('ugid').number.count().reset_index()
    if days == 30:
        callcnt_agg_wkd.columns = ['ugid', 'last1m_callcnt_agg_wkd']
        feature01 = pd.merge(feature01, callcnt_agg_wkd, how = 'left', on = 'ugid')
        calltime_agg_wkd.columns = ['ugid', 'last1m_calltime_agg_wkd']
        feature01 = pd.merge(feature01, calltime_agg_wkd, how = 'left', on = 'ugid')
        clncnt_dtotal_wkd.columns = ['ugid', 'last1m_clncnt_dtotal_wkd']
        feature01 = pd.merge(feature01, clncnt_dtotal_wkd, how = 'left', on = 'ugid')
        feature01['last1m_calltime_avg_wkd'] = feature01['last1m_calltime_agg_wkd']/feature01['last1m_callcnt_agg_wkd']
    elif days == 60:
        callcnt_agg_wkd.columns = ['ugid', 'last2m_callcnt_agg_wkd']
        feature01 = pd.merge(feature01, callcnt_agg_wkd, how = 'left', on = 'ugid')
        calltime_agg_wkd.columns = ['ugid', 'last2m_calltime_agg_wkd']
        feature01 = pd.merge(feature01, calltime_agg_wkd, how = 'left', on = 'ugid')
        clncnt_dtotal_wkd.columns = ['ugid', 'last2m_clncnt_dtotal_wkd']
        feature01 = pd.merge(feature01, clncnt_dtotal_wkd, how = 'left', on = 'ugid')
        feature01['last2m_calltime_avg_wkd'] = feature01['last2m_calltime_agg_wkd']/feature01['last2m_callcnt_agg_wkd']
    return(feature01)
        

In [37]:
def getWeekBaseFeature(feature01,df,days):
    '''
    输入一个dataframe，生成所有weekbase特征，包括活跃天数，通话总时长，通话总次数，平均通话时长，去重号码数
    '''
    df_weekday = df[df.active_day.apply(datetime.weekday) <= 4].reset_index(drop=True)
    df_weekend = df[df.active_day.apply(datetime.weekday) > 4  ].reset_index(drop=True)
    
    feature01 = weekday_active_days(feature01,df_weekday,days)     #工作日活跃天数  2*1
    feature01 = weekend_active_days(feature01,df_weekend,days)      #休息日活跃天数 2*1
    feature01 = weekdayBaseSDK(feature01,df_weekday,days)          #工作日          2*4
    feature01 = weekendBaseSDK(feature01,df_weekend,days)          #休息日          2*4
    return(feature01)

# 特征提取入口

* 在时间窗口内用全部数据进行计算
    * 过去三十天访问注册页面次数（eg）
* 在时间窗口内定义不同的时间组别：
    * 休息日和工作日
        * 过去三十天工作日访问注册页面次数（eg）
        * 过去三十天休息日访问注册页面次数（eg）
    * 每天的不同时间段
        * 过去三十天在凌晨0-4点访问注册页面次数（eg）timePeriod0-5
        * 过去三十天在12-16点访问注册页面次数（eg）

In [38]:
con01 = pd.read_csv('./data_mid/20171122_data_con01.201706_201708.csv',encoding="utf-8")
con01 = con01[['ugid', 'phone']]

In [39]:
#生成并保存近两个月的用户通话数据
df_60=cal01[(cal01.date + timedelta(days=60) > cal01.last_decision_tm) &  (cal01.date<cal01.last_decision_tm) ].reset_index(drop=True)

In [41]:
df_60.to_csv('df_60.csv', header = True, index=False, encoding="utf-8")

In [42]:
#生成并保存近一个月的用户通话数据
df_30=cal01[(cal01.date + timedelta(days=30) > cal01.last_decision_tm) &  (cal01.date<cal01.last_decision_tm) ].reset_index(drop=True)

In [44]:
df_30.to_csv('df_30.csv', header = True, index=False, encoding="utf-8")

In [45]:
#生成用户ugid和通讯录的对应关系表的数据准备
con = con01
con['phone'] = con['phone'].replace(np.nan,'-1').astype('str')
con['phone'] = con['phone'].replace(np.inf,'-1').astype('str')
con['phone'] = con['phone'].replace('null','-1').astype('str')
con.shape

(288276, 2)

In [46]:
con = con[con['phone'] != '-1']
con.shape

(286506, 2)

In [47]:
con01 = con

In [48]:
def contacts(x):
    data = con01[con01['ugid'] == x]
    thelist = data['phone'].tolist()
    return(thelist)

In [49]:
#生成并保存用户ugid和通讯录的对应关系表
df_contacts = feature01
df_contacts['contacts'] = df_contacts.ugid.apply(lambda x:contacts(x))
df_contacts.to_csv('df_contacts_201701_201705.csv', header = True, index=False)

df_contacts = pd.read_csv('df_contacts_201701_201705.csv')

In [51]:
df_contacts = df_contacts[['ugid', 'contacts']]

In [53]:
#定义基于每个ugid中每个number的统计特征（主要是为了减小统计量，针对每行统计工作量太大，
#针对每个ugid中的每个number进行合并统计后能极大的减小统计工作量）
def ugid_number_feature(df,days):
    df_in = df[df['types'] == 1]                           #被叫
    df_out = df[df['types'] == 2]                           #主叫
    df_other = df[df['types'] == 3]                           #其他
    if days == 30:
        callcnt_agg_ctct = df.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数
        callcnt_agg_ctct.columns = ['ugid', 'number', 'last1m_callcnt_agg_ctct']
        callcnt_agg_ctct_in = df_in.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数-被叫
        callcnt_agg_ctct_in.columns = ['ugid', 'number', 'last1m_callcnt_agg_ctct_in']
        callcnt_agg_ctct_out = df_out.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数-被叫
        callcnt_agg_ctct_out.columns = ['ugid', 'number', 'last1m_callcnt_agg_ctct_out']
        callcnt_agg_ctct_other = df_other.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数-被叫
        callcnt_agg_ctct_other.columns = ['ugid', 'number', 'last1m_callcnt_agg_ctct_other']
        
        calltime_agg_ctct = df.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长
        calltime_agg_ctct.columns = ['ugid', 'number', 'last1m_calltime_agg_ctct']
        calltime_agg_ctct_in = df_in.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长-被叫
        calltime_agg_ctct_in.columns = ['ugid', 'number', 'last1m_calltime_agg_ctct_in']
        calltime_agg_ctct_out = df_out.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长-被叫
        calltime_agg_ctct_out.columns = ['ugid', 'number', 'last1m_calltime_agg_ctct_out']
        calltime_agg_ctct_other = df_other.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长-被叫
        calltime_agg_ctct_other.columns = ['ugid', 'number', 'last1m_calltime_agg_ctct_other']
        
        
    elif days == 60:
        callcnt_agg_ctct = df.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数
        callcnt_agg_ctct.columns = ['ugid', 'number', 'last2m_callcnt_agg_ctct']
        callcnt_agg_ctct_in = df_in.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数-被叫
        callcnt_agg_ctct_in.columns = ['ugid', 'number', 'last2m_callcnt_agg_ctct_in']
        callcnt_agg_ctct_out = df_out.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数-被叫
        callcnt_agg_ctct_out.columns = ['ugid', 'number', 'last2m_callcnt_agg_ctct_out']
        callcnt_agg_ctct_other = df_other.groupby(['ugid','number']).duration.count().reset_index()     #通话总次数-被叫
        callcnt_agg_ctct_other.columns = ['ugid', 'number', 'last2m_callcnt_agg_ctct_other']
        
        calltime_agg_ctct = df.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长
        calltime_agg_ctct.columns = ['ugid', 'number', 'last2m_calltime_agg_ctct']
        calltime_agg_ctct_in = df_in.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长-被叫
        calltime_agg_ctct_in.columns = ['ugid', 'number', 'last2m_calltime_agg_ctct_in']
        calltime_agg_ctct_out = df_out.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长-被叫
        calltime_agg_ctct_out.columns = ['ugid', 'number', 'last2m_calltime_agg_ctct_out']
        calltime_agg_ctct_other = df_other.groupby(['ugid','number']).duration.sum().reset_index()     #通话总时长-被叫
        calltime_agg_ctct_other.columns = ['ugid', 'number', 'last2m_calltime_agg_ctct_other']
        
    df_in_ = df_in[['ugid','number']].drop_duplicates().reset_index()
    df_in_ = df_in_[['ugid','number']]
    df_in_['number_in'] = 1
    df_out_ = df_out[['ugid','number']].drop_duplicates().reset_index()
    df_out_ = df_out_[['ugid','number']]
    df_out_['number_out'] = 1
    df_other_ = df_other[['ugid','number']].drop_duplicates().reset_index()
    df_other_ = df_other_[['ugid','number']]   
    df_other_['number_other'] = 1
        
    df_ugid_number = callcnt_agg_ctct                                                          #df_60_ugid_number保存所有生成的特征
    df_ugid_number = pd.merge(df_ugid_number, callcnt_agg_ctct_in, how = 'left', on = ['ugid', 'number'])
    df_ugid_number = pd.merge(df_ugid_number, callcnt_agg_ctct_out, how = 'left', on = ['ugid', 'number'])
    df_ugid_number = pd.merge(df_ugid_number, callcnt_agg_ctct_other, how = 'left', on = ['ugid', 'number'])
    
    df_ugid_number = pd.merge(df_ugid_number, calltime_agg_ctct, how = 'left', on = ['ugid', 'number'])
    df_ugid_number = pd.merge(df_ugid_number, calltime_agg_ctct_in, how = 'left', on = ['ugid', 'number'])
    df_ugid_number = pd.merge(df_ugid_number, calltime_agg_ctct_out, how = 'left', on = ['ugid', 'number'])
    df_ugid_number = pd.merge(df_ugid_number, calltime_agg_ctct_other, how = 'left', on = ['ugid', 'number'])
    
    df_ugid_number = pd.merge(df_ugid_number, df_in_, how = 'left', on = ['ugid', 'number'])
    df_ugid_number = pd.merge(df_ugid_number, df_out_, how = 'left', on = ['ugid', 'number'])
    df_ugid_number = pd.merge(df_ugid_number, df_other_, how = 'left', on = ['ugid', 'number'])
    
    return(df_ugid_number)


In [54]:
#生成基于每个ugid中每个number的统计特征
df_60_ugid_number = ugid_number_feature(df_60,60)
df_60_ugid_number.to_csv('df_60_ugid_number_201701_201705.csv', header = True, index=False)

In [55]:
#生成基于每个ugid中每个number的统计特征
df_30_ugid_number = ugid_number_feature(df_30,30)
df_30_ugid_number.to_csv('df_30_ugid_number_201701_201705.csv', header = True, index=False)

#读取基于每个ugid中每个number的统计特征，即每个每个ugid中每个number的主叫次数，被叫次数，通话时长等特征
df_60_ugid_number = pd.read_csv('/df_60_ugid_number.csv')
df_30_ugid_number = pd.read_csv('df_30_ugid_number.csv')

In [59]:
#将基于每个ugid中每个number的统计特征和通讯录关联上，并标记每个number是否在通讯录中
df_60_ugid_number_contacts = pd.merge(df_60_ugid_number, df_contacts, how = 'left', on = 'ugid')
df_30_ugid_number_contacts = pd.merge(df_30_ugid_number, df_contacts, how = 'left', on = 'ugid')

In [60]:
#定义标记一个number是否在通讯录中的函数
def in_address_book_1(row):
    thelist = row['contacts']
    if row['number'] in thelist:
        if row['number'] != np.nan:
            return(True)
    else:
        return(False)

In [61]:
#标记number是否在通讯录中，并保存
df_60_ugid_number_contacts['in_address_book'] = df_60_ugid_number_contacts.apply(lambda row:in_address_book_1(row), axis=1)
df_60_ugid_number_contacts.to_csv('df_60_ugid_number_contacts_201701_201705.csv', header = True, index=False)     #60天内的针对ugid和number统计的行为数据，标记了是否包含在通讯录中

df_30_ugid_number_contacts['in_address_book'] = df_30_ugid_number_contacts.apply(lambda row:in_address_book_1(row), axis=1)
df_30_ugid_number_contacts.to_csv('df_30_ugid_number_contacts_201701_201705.csv', header = True, index=False)     #30天内的针对ugid和number统计的行为数据，标记了是否包含在通讯录中

#读取保存的标记了ugid和number是否包含在通讯录中的数据
df_60_ugid_number_contacts = pd.read_csv('df_60_ugid_number_contacts.csv')
df_30_ugid_number_contacts = pd.read_csv('df_30_ugid_number_contacts.csv')

In [64]:
#定义统计topN（次数）与通讯录中电话号码的重合数，基于每个ugid统计的
def isTopN_num(row,N,days):
    if days == 30:
        data = row.sort_index(axis=0, by='last1m_callcnt_agg_ctct', ascending=False).reset_index()
    elif days == 60:
        data = row.sort_index(axis=0, by='last2m_callcnt_agg_ctct', ascending=False).reset_index()
    
    thecontacts = data['contacts'][0]
    
    if data.shape[0] > N:
        data = data[:N][:]
    thelist = data['number'].tolist()    #从全部数据中抽出topN（次数）相关部分，并转换成list
    num = 0
    for item in thelist:
        if item in thecontacts:
            num = num + 1
    return(num)

In [65]:
#定义统计topN（时长）与通讯录中电话号码的重合数，基于每个ugid统计的
def isTopN_dur(row,N,days):
    if days == 30:
        data = row.sort_index(axis=0, by='last1m_calltime_agg_ctct', ascending=False).reset_index()
    elif days == 60:
        data = row.sort_index(axis=0, by='last2m_calltime_agg_ctct', ascending=False).reset_index()
    
    thecontacts = data['contacts'][0]
    
    if data.shape[0] > N:
        data = data[:N][:]
    thelist = data['number'].tolist()
    num = 0
    for item in thelist:
        if item in thecontacts:
            num = num + 1
    return(num)

In [66]:
#生成统计时间内topN(次数、时长)与通讯录号码的重合数特征
def getTopNBaseFeature(feature01,df,days):
    ugid = df[['ugid']].drop_duplicates().reset_index()
    ugid = ugid[['ugid']]
    if days == 60:
        #topN次数
        ugid['last2m_callcnt_ctctnum_cln_top10'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,10,60))
        ugid['last2m_callcnt_ctctnum_cln_top20'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,20,60))
        ugid['last2m_callcnt_ctctnum_cln_top30'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,30,60))
        ugid['last2m_callcnt_ctctnum_cln_top40'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,40,60))
        ugid['last2m_callcnt_ctctnum_cln_top50'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,50,60))
        #topN时长
        ugid['last2m_calltime_ctctnum_cln_top10'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,10,60))
        ugid['last2m_calltime_ctctnum_cln_top20'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,20,60))
        ugid['last2m_calltime_ctctnum_cln_top30'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,30,60))
        ugid['last2m_calltime_ctctnum_cln_top40'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,40,60))
        ugid['last2m_calltime_ctctnum_cln_top50'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,50,60))
        feature01 = pd.merge(feature01,ugid,how = 'left', on = 'ugid')

    elif days == 30:
        #topN次数
        ugid['last1m_callcnt_ctctnum_cln_top10'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,10,30))
        ugid['last1m_callcnt_ctctnum_cln_top20'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,20,30))
        ugid['last1m_callcnt_ctctnum_cln_top30'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,30,30))
        ugid['last1m_callcnt_ctctnum_cln_top40'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,40,30))
        ugid['last1m_callcnt_ctctnum_cln_top50'] = df.groupby('ugid').apply(lambda row:isTopN_num(row,50,30))
        #topN时长
        ugid['last1m_calltime_ctctnum_cln_top10'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,10,30))
        ugid['last1m_calltime_ctctnum_cln_top20'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,20,30))
        ugid['last1m_calltime_ctctnum_cln_top30'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,30,30))
        ugid['last1m_calltime_ctctnum_cln_top40'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,40,30))
        ugid['last1m_calltime_ctctnum_cln_top50'] = df.groupby('ugid').apply(lambda row:isTopN_dur(row,50,30))
        feature01 = pd.merge(feature01,ugid,how = 'left', on = 'ugid')
    
    return(feature01)

In [70]:
def getContactsBaseFeature(feature01,df,days):
    df = df[df['in_address_book'] == True]
    ugid = df[['ugid']].drop_duplicates().reset_index()
    ugid = ugid[['ugid']]
    if days == 30:
        #通话总次数
        last1m_callcnt_agg_ctct = df.groupby('ugid').last1m_callcnt_agg_ctct.sum().reset_index()
        last1m_callcnt_agg_ctct_in = df.groupby('ugid').last1m_callcnt_agg_ctct_in.sum().reset_index()
        last1m_callcnt_agg_ctct_out = df.groupby('ugid').last1m_callcnt_agg_ctct_out.sum().reset_index()
        last1m_callcnt_agg_ctct_other = df.groupby('ugid').last1m_callcnt_agg_ctct_other.sum().reset_index()
        
        #通话总时长
        last1m_calltime_agg_ctct = df.groupby('ugid').last1m_calltime_agg_ctct.sum().reset_index()
        last1m_calltime_agg_ctct_in = df.groupby('ugid').last1m_calltime_agg_ctct_in.sum().reset_index()
        last1m_calltime_agg_ctct_out = df.groupby('ugid').last1m_calltime_agg_ctct_out.sum().reset_index()
        last1m_calltime_agg_ctct_other = df.groupby('ugid').last1m_calltime_agg_ctct_other.sum().reset_index()
        
        #电话号码重合数
        last1m_overlapcnt_ctctnum_agg = df.groupby('ugid').number.count().reset_index()
        last1m_overlapcnt_ctctnum_agg.columns = ['ugid','last1m_overlapcnt_ctctnum_agg']
        last1m_overlapcnt_ctctnum_in = df.groupby('ugid').number_in.sum().reset_index()
        last1m_overlapcnt_ctctnum_in.columns = ['ugid','last1m_overlapcnt_ctctnum_in']
        last1m_overlapcnt_ctctnum_out = df.groupby('ugid').number_out.sum().reset_index()
        last1m_overlapcnt_ctctnum_out.columns = ['ugid','last1m_overlapcnt_ctctnum_out']
        last1m_overlapcnt_ctctnum_other = df.groupby('ugid').number_other.sum().reset_index()
        last1m_overlapcnt_ctctnum_other.columns = ['ugid','last1m_overlapcnt_ctctnum_other']
        
        
        feature01 = pd.merge(feature01,last1m_callcnt_agg_ctct,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_callcnt_agg_ctct_in,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_callcnt_agg_ctct_out,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_callcnt_agg_ctct_other,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_calltime_agg_ctct,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_calltime_agg_ctct_in,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_calltime_agg_ctct_out,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_calltime_agg_ctct_other,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_overlapcnt_ctctnum_agg,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_overlapcnt_ctctnum_in,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_overlapcnt_ctctnum_out,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last1m_overlapcnt_ctctnum_other,how = 'left',on = 'ugid')
        
        feature01['last1m_calltime_avg_ctct'] = feature01['last1m_calltime_agg_ctct'] / feature01['last1m_callcnt_agg_ctct']
        feature01['last1m_calltime_avg_ctct_in'] = feature01['last1m_calltime_agg_ctct_in'] / feature01['last1m_callcnt_agg_ctct_in']
        feature01['last1m_calltime_avg_ctct_out'] = feature01['last1m_calltime_agg_ctct_out'] / feature01['last1m_callcnt_agg_ctct_out']
        feature01['last1m_calltime_avg_ctct_other'] = feature01['last1m_calltime_agg_ctct_other'] / feature01['last1m_callcnt_agg_ctct_other']
        
    elif  days == 60:
        last2m_callcnt_agg_ctct = df.groupby('ugid').last2m_callcnt_agg_ctct.sum().reset_index()
        last2m_callcnt_agg_ctct_in = df.groupby('ugid').last2m_callcnt_agg_ctct_in.sum().reset_index()
        last2m_callcnt_agg_ctct_out = df.groupby('ugid').last2m_callcnt_agg_ctct_out.sum().reset_index()
        last2m_callcnt_agg_ctct_other = df.groupby('ugid').last2m_callcnt_agg_ctct_other.sum().reset_index()
        
        last2m_calltime_agg_ctct = df.groupby('ugid').last2m_calltime_agg_ctct.sum().reset_index()
        last2m_calltime_agg_ctct_in = df.groupby('ugid').last2m_calltime_agg_ctct_in.sum().reset_index()
        last2m_calltime_agg_ctct_out = df.groupby('ugid').last2m_calltime_agg_ctct_out.sum().reset_index()
        last2m_calltime_agg_ctct_other = df.groupby('ugid').last2m_calltime_agg_ctct_other.sum().reset_index()
        
        last2m_overlapcnt_ctctnum_agg = df.groupby('ugid').number.count().reset_index()
        last2m_overlapcnt_ctctnum_agg.columns = ['ugid','last2m_overlapcnt_ctctnum_agg']
        last2m_overlapcnt_ctctnum_in = df.groupby('ugid').number_in.sum().reset_index()
        last2m_overlapcnt_ctctnum_in.columns = ['ugid','last2m_overlapcnt_ctctnum_in']
        last2m_overlapcnt_ctctnum_out = df.groupby('ugid').number_out.sum().reset_index()
        last2m_overlapcnt_ctctnum_out.columns = ['ugid','last2m_overlapcnt_ctctnum_out']
        last2m_overlapcnt_ctctnum_other = df.groupby('ugid').number_other.sum().reset_index()
        last2m_overlapcnt_ctctnum_other.columns = ['ugid','last2m_overlapcnt_ctctnum_other']
       
        
        feature01 = pd.merge(feature01,last2m_callcnt_agg_ctct,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_callcnt_agg_ctct_in,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_callcnt_agg_ctct_out,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_callcnt_agg_ctct_other,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_calltime_agg_ctct,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_calltime_agg_ctct_in,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_calltime_agg_ctct_out,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_calltime_agg_ctct_other,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_overlapcnt_ctctnum_agg,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_overlapcnt_ctctnum_in,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_overlapcnt_ctctnum_out,how = 'left',on = 'ugid')
        feature01 = pd.merge(feature01,last2m_overlapcnt_ctctnum_other,how = 'left',on = 'ugid')
        feature01['last2m_calltime_avg_ctct'] = feature01['last2m_calltime_agg_ctct'] / feature01['last2m_callcnt_agg_ctct']
        feature01['last2m_calltime_avg_ctct_in'] = feature01['last2m_calltime_agg_ctct_in'] / feature01['last2m_callcnt_agg_ctct_in']
        feature01['last2m_calltime_avg_ctct_out'] = feature01['last2m_calltime_agg_ctct_out'] / feature01['last2m_callcnt_agg_ctct_out']
        feature01['last2m_calltime_avg_ctct_other'] = feature01['last2m_calltime_agg_ctct_other'] / feature01['last2m_callcnt_agg_ctct_other']

    return(feature01)

In [71]:
def getPeriodBetweenApplyCache(feature01,df,days):
    '''
    输入一个dataframe，计算统计时间内申请时间与通话记录数据爬取的时间差
    '''
    # 先排序
    df.sort_values(by=['ugid','ctime'],ascending=[True,False],inplace=True)
    
    # 再去重
    df = df.drop_duplicates(['ugid'])
    
    #保留必须的字段
    df = df[['ugid','ctime','last_decision_tm']]
    df['period_between_apply_cache-y'] = df['last_decision_tm'] - df['ctime']
    df = df[['ugid','period_between_apply_cache-y']]
    feature01 = pd.merge(feature01,df,how = 'left',on = 'ugid')
    
    return(feature01)

特征提取入口
在时间窗口内用全部数据进行计算
过去三十天访问注册页面次数（eg）
在时间窗口内定义不同的时间组别：
休息日和工作日
过去三十天工作日访问注册页面次数（eg）
过去三十天休息日访问注册页面次数（eg）
每天的不同时间段
过去三十天在凌晨0-4点访问注册页面次数（eg）timePeriod0-5
过去三十天在12-16点访问注册页面次数（eg）

In [72]:
#针对不同的appid类型，对数据进行分类
df=cal01[(cal01.date + timedelta(days=30) > cal01.last_decision_tm) &  (cal01.date<cal01.last_decision_tm) ].reset_index(drop=True)

In [73]:
test = df_30[['ugid','number','appid']]
df_30_ugid_number_contacts =  pd.merge(df_30_ugid_number_contacts, test, how = 'left', on = ['ugid','number'])

In [74]:
#针对不同的appid类型，对数据进行分类
df=cal01[(cal01.date+ timedelta(days=60) > cal01.last_decision_tm) &  (cal01.date<cal01.last_decision_tm) ].reset_index(drop=True)

In [75]:
test = df_60[['ugid','number','appid']]
df_60_ugid_number_contacts =  pd.merge(df_60_ugid_number_contacts, test, how = 'left', on = ['ugid','number'])

In [76]:
#针对不同的appid类型，提取过去一个月的特征
feature01 = df_30[['ugid']].drop_duplicates().reset_index()     #注意每次运行时都需要改变数据集
feature01 = feature01[['ugid']]
feature01 = getDayBasefeature(feature01,df_30,days=30)
feature01 = getWeekBaseFeature(feature01,df_30,days=30)

feature01 = getTopNBaseFeature(feature01,df_30_ugid_number_contacts,days=30)
feature01 = getContactsBaseFeature(feature01,df_30_ugid_number_contacts,days=30)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  after removing the cwd from sys.path.


In [ ]:
feature01_ = feature01

In [77]:
#针对不同的appid类型，提取过去两个月的特征

feature01 = getDayBasefeature(feature01,df_60,days=60)
feature01 = getWeekBaseFeature(feature01,df_60,days=60)
feature01 = getPeriodBetweenApplyCache(feature01,df_60,days=60)

feature01 = getTopNBaseFeature(feature01,df_60_ugid_number_contacts,days=60)
feature01 = getContactsBaseFeature(feature01,df_60_ugid_number_contacts,days=60)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  


In [79]:
feature01.shape

(199, 120)

In [80]:
feature01.to_csv('./data_last/20171123_feature_cal01_201706_201708.csv', header = True, index=False)